In [46]:
import pickle
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [47]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
df = pd.read_csv('modified_data.csv', encoding='ISO-8859-1')

In [50]:
df.head()

,Unnamed: 0,target,id,date,flag,user,text,stemmed_content
0,0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [51]:
twitter_data = pd.read_excel('tweets1.xlsx', names=['tweets'])
twitter_data.head()

,tweets
0,Ready to embrace the slimy adventure? Share yo...
1,This is why we #bitcoin
2,Motokos are coming to Bitcoin Ordinals! All as...
3,A family member called and said: you should co...
4,"JUST IN: Fidelity, the world's third-largest a..."


In [52]:
#stemming
port_stem = PorterStemmer()

def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [53]:
twitter_data['stemmed_tweets'] = twitter_data['tweets'].apply(stemming)

In [54]:
twitter_data.head()

,tweets,stemmed_tweets
0,Ready to embrace the slimy adventure? Share yo...,readi embrac slimi adventur share excit chanc ...
1,This is why we #bitcoin,bitcoin
2,Motokos are coming to Bitcoin Ordinals! All as...,motoko come bitcoin ordin asset fulli inscrib ...
3,A family member called and said: you should co...,famili member call said consid sell bitcoin pr...
4,"JUST IN: Fidelity, the world's third-largest a...",fidel world third largest asset manag file spo...


In [55]:
stemmed_content = df['stemmed_content'].values
sentiment = df['target'].values
stemmed_tweets = twitter_data['stemmed_tweets'].values

In [56]:
stemmed_train, stemmed_test, sentiment_train, sentiment_test = train_test_split(stemmed_content, sentiment, test_size=0.2, stratify=sentiment, random_state=2)

In [57]:
print(stemmed_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [59]:
#converting to numerical data

vectorizer = TfidfVectorizer()

stemmed_train = vectorizer.fit_transform(stemmed_train.astype('U'))
stemmed_test = vectorizer.transform(stemmed_test.astype('U'))
stemmed_tweets = vectorizer.transform(stemmed_tweets.astype('U'))


In [60]:
print(stemmed_tweets)

  (0, 443253)	0.20082047956611626
  (0, 442911)	0.14819212705526258
  (0, 411790)	0.16339325465095247
  (0, 393756)	0.2003720000602922
  (0, 379705)	0.1986255424293775
  (0, 371971)	0.21491038918702599
  (0, 371923)	0.2816869946267881
  (0, 362324)	0.16559068970711102
  (0, 338944)	0.19462899592208846
  (0, 334593)	0.1336660066635045
  (0, 303498)	0.21217246755708832
  (0, 234918)	0.09285908286127248
  (0, 232891)	0.12711873916271532
  (0, 203617)	0.18403843416161067
  (0, 201210)	0.1632728895060458
  (0, 179523)	0.22263518846329508
  (0, 158791)	0.3179067934165423
  (0, 125698)	0.23396932142099622
  (0, 125641)	0.1364506887719642
  (0, 118378)	0.2398336911330484
  (0, 66648)	0.3377030900967761
  (0, 39566)	0.19578484038141614
  (0, 28865)	0.13662887044059246
  (0, 4889)	0.19957160758591522
  (2, 427283)	0.1283640958307027
  :	:
  (22, 170357)	0.2598749993797496
  (22, 100890)	0.3501308323823688
  (22, 45496)	0.37959648393849715
  (23, 400375)	0.12214936424772112
  (23, 387014)	0.19656

In [61]:
#Logistic regression ML model

model = LogisticRegression(max_iter=1000)

In [62]:
model.fit(stemmed_train, sentiment_train)

LogisticRegression(max_iter=1000)

In [63]:
stemmed_tweets_prediction = model.predict(stemmed_tweets)
print(stemmed_tweets_prediction)

[1 1 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 0]


In [66]:
popular_crypto = ['bitcoin', 'ethereum', 'tether USDT', 'BNB', 'solana']
for i in range(5):
    print("%s : %i" % (popular_crypto[i], (stemmed_tweets_prediction[5*i] + stemmed_tweets_prediction[5*i + 1] + stemmed_tweets_prediction[5*i + 2] + stemmed_tweets_prediction[5*i + 3])))

bitcoin : 2
ethereum : 2
tether USDT : 1
BNB : 3
solana : 2
